In [428]:
!pip install heartpy -q
!pip install mne -q
!pip install neurokit2 -q

^C


In [505]:
import json
from os import listdir
from tqdm import tqdm

import numpy as np
import pandas as pd
import mne
import neurokit2 as nk
import heartpy as hp

file_path = 'pairs.json'

with open(file_path, 'r') as json_file:
    loaded_dict = json.load(json_file)

files = np.array([f for f in listdir('Data_upd/') if '.edf' in f])

### Code for 8 files (grouped by videos)

In [ ]:
broken = []
result_m = {'C1':{}, 'C2':{}, 'F1':{}, 'F2':{}, 'V1':{}, 'V2':{}, 'V3':{}, 'V4':{}}
result_b = {'C1':{}, 'C2':{}, 'F1':{}, 'F2':{}, 'V1':{}, 'V2':{}, 'V3':{}, 'V4':{}}

for idx, f in tqdm(enumerate(files)):
    print(f'====PROCEISSING {f}====')
    for idx_v, video in enumerate(['C1', 'C2', 'F1', 'F2', 'V1', 'V2', 'V3','V4']):
        print(f'-----{video}-----')
        data = mne.io.read_raw_edf('Data_upd/' + f, preload=True, misc=('Zygom', 'Corr', 'EDA', 'Pulse', 'Mark'))
        data = data.pick_channels(['Pulse'])
        
        start, end = loaded_dict[f][idx_v]

        ppg = data[0][0][0][int(start):int(end)]
        ppg_b = data[0][0][0][(int(start)-10000):int(start)]

        ppg = nk.ppg_clean(ppg, sampling_rate=500)
        ppg_b = nk.ppg_clean(ppg_b, sampling_rate=500)

        wd, m = hp.process(ppg, sample_rate=500, calc_freq=True)
        wd_b, m_b = hp.process(ppg_b, sample_rate=500, calc_freq=True)

        try:
            wd, m = hp.process(ppg, sample_rate=500, calc_freq=True)
            wd_b, m_b = hp.process(ppg_b, sample_rate=500, calc_freq=True)
        except:
            print('ACHTUNG!')
            broken.append([f, idx_v])
            continue

        result_m[video][f] = m
        result_b[video][f] = m_b

### Code for 60 files (grouped by participants)

In [ ]:
broken = []
result = {f:{} for f in files}

for idx, f in tqdm(enumerate(files)):
    print(f'====PROCEISSING {f}====')
    for idx_v, video in enumerate(['C1', 'C2', 'F1', 'F2', 'V1', 'V2', 'V3','V4']):
        print(f'-----{video}-----')
        data = mne.io.read_raw_edf('Data_upd/' + f, preload=True, misc=('Zygom', 'Corr', 'EDA', 'Pulse', 'Mark'))
        data = data.pick_channels(['Pulse'])
        
        start, end = loaded_dict[f][idx_v]

        ppg = data[0][0][0][int(start):int(end)]
        ppg_b = data[0][0][0][(int(start)-10000):int(start)]

        ppg = nk.ppg_clean(ppg, sampling_rate=500)
        ppg_b = nk.ppg_clean(ppg_b, sampling_rate=500)

        wd, m = hp.process(ppg, sample_rate=500, calc_freq=True)
        wd_b, m_b = hp.process(ppg_b, sample_rate=500, calc_freq=True)

        result[f]['baseline_' + video] = m_b
        result[f][video] = m

In [559]:
for f in tqdm(files):
    pd.DataFrame(result[f]).T.to_csv(f'hrv/{f.split(".")[0]}.csv')

100%|██████████| 60/60 [00:00<00:00, 289.98it/s]
